In [28]:
import sys
sys.path.append('..\.')
from utils.qSLP import qSLP
from utils.Utils import get_params, parity
from utils.import_data import get_dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import COBYLA
from qiskit import Aer, QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.circuit.library import RealAmplitudes
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier

from sklearn.metrics import accuracy_score

seed = 42
max_iter = 150

In [17]:
def get_qNNC(version = 1):
    if version == 1:
        feature_map = ZZFeatureMap(feature_dimension=2,
                            reps=1, entanglement='linear')
        ansatz = RealAmplitudes(2, reps=1)

    elif version == 2:
        feature_map = ZFeatureMap(feature_dimension=2,
                                reps=1)
        ansatz = RealAmplitudes(2, reps=2)

    else:
        raise Exception("Version not implemented")
    
    return feature_map, ansatz


## Set the specification for the model

In [23]:
# Change version to change the qnnc model
version = 1

# Change to change the dataset
dataset = "iris01"

# model required variable
tot_qubit = 2
output_shape = 2


## Training of the model
We train the qSLP model starting from given starting points


In [30]:

model_name = f"qNNC_v{version}"

#function that returns the best parameters for a given model
starting_points = get_params(model_name, dataset, "starting_points", "../results/training/file_result.txt")


### Obtain the dataset
The dataset is processed through the use a PCA in order to use only two high descriptive features. 


In [25]:
X_train, X_test, Y_train, Y_test = get_dataset(dataset)

[0.90539269 0.07445563]
98.0% of total variance is explained by 2 principal components


### Set the optimizer and the quantum instance


In [26]:
optimizer = COBYLA(maxiter=max_iter, tol=0.01, disp=False)
qinstance = QuantumInstance(Aer.get_backend('aer_simulator'),seed_simulator=seed,seed_transpiler=seed, shots=1024)
qinstance.backend.set_option("seed_simulator", seed)


### The model 
Build the model with the chosen parameters.


In [31]:
feature_map, ansatz = get_qNNC(1)
interpreter = parity 
qc = QuantumCircuit(tot_qubit)
qc.append(feature_map, range(tot_qubit))
qc.append(ansatz, range(tot_qubit))

objective_func_vals = []
def callback_values(weights, obj_func_eval):
    objective_func_vals.append(obj_func_eval)

circuit_qnn = CircuitQNN(circuit=qc,
                        input_params=feature_map.parameters,
                        weight_params=ansatz.parameters,
                        interpret=interpreter,
                        output_shape=output_shape,
                        quantum_instance=qinstance)

circuit_classifier = NeuralNetworkClassifier(neural_network=circuit_qnn,
                                        optimizer=optimizer,
                                        callback=callback_values, 
                                        warm_start=True,
                                        initial_point = starting_points )




## Training


In [33]:
circuit_classifier.fit(X_train, Y_train)


In [36]:
train_score = circuit_classifier.score(X_train, Y_train)
test_score = circuit_classifier.score(X_test, Y_test)


In [38]:
print(train_score)
print(test_score)

0.9
0.95


In [39]:
ending_points = circuit_classifier._fit_result[0]


In [40]:
get_params(model_name, dataset, "ending_points", "../results/training/file_result.txt")

[0.09390477293309543,
 1.5886292417212462,
 -0.9110389376309079,
 0.4951799698639165]

In [41]:
ending_points

array([ 0.09390477,  1.58862924, -0.91103894,  0.49517997])